In [1]:
run_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\representativity\\models"
data_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\representativity\\data"
train_data_string = ["X_rep_training_data.csv.npy",
                     "Y_rep_training_data.csv.npy"]
validation_data_string = ["X_rep_validation_data.csv.npy",
                          "Y_rep_validation_data.csv.npy"]

In [2]:
import tensorflow
import os

In [3]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.1
Scikit-Learn 0.24.1
GPU is available


In [4]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from matplotlib import pyplot as plt
from IPython.display import clear_output
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
%matplotlib inline
# Load the TensorBoard notebook extension.
%load_ext tensorboard
#from keras_radam import RAdam
import pandas as pd
import numpy as np
import time

In [5]:
import tensorflow as tf
print(keras.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.4-tf
Num GPUs Available:  1


In [6]:
import kerastuner
from kerastuner.tuners import RandomSearch, BayesianOptimization 
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
#data_ = np.loadtxt("FNS_GA_Data_Runs_11_12_Paper_2.csv", delimiter=",")
#X = data_[:,0:150]
os.chdir(data_directory)
X_train = np.load(train_data_string[0])
Y_train = np.load(train_data_string[1])


#data_test = np.loadtxt("FNS_GA_Run_13_Data_Test.csv", delimiter=",")
X_test = np.load(validation_data_string[0])
Y_test = np.load(validation_data_string[1])

#X_test = np.load("test_2x2_fns_run_test_dat.npy")

In [ ]:
print(X_train[0])
print(Y_train[0])

In [ ]:
#def build_model():
#    model = Sequential()
#    model.add(Conv3D(24, kernel_size=(2, 2, 2),
#                     input_shape=(60, 2, 2, 4), padding='same', data_format="channels_last", strides = (1, 1, 1)))
#    model.add(LeakyReLU(alpha=0.3))
#    model.add(BatchNormalization())
#    ##model.add(Conv3D(32, (3, 3, 5), activation=LeakyReLU(alpha=0.3), padding='same'))
#    for _ in range(5):
#        model.add(Conv3D(24, (5, 2, 2), padding='same'))
#        model.add(LeakyReLU(alpha=0.3))
#        model.add(BatchNormalization())
#    model.add(MaxPooling3D(pool_size=(6, 2, 2)))
#    model.add(Flatten())
#    model.add(BatchNormalization())
#    model.add(Dense(128))
#    model.add(LeakyReLU(alpha=0.3))
#    model.add(BatchNormalization())
#    model.add(Dense(128))
#    model.add(LeakyReLU(alpha=0.3))
#    model.add(Dropout(0.25))
#    model.add(Dense(1, activation='linear'))
#
#    ### Building the optimizer object
#    #sgd = optimizers.SGD(lr = 0.1) # 1.0 0.0001 0.01
#    opt = optimizers.Adam(lr = 0.005) # 1.0 0.0001 0.01
#    ### Adding the loss function and optimizer to the model
#    mcp_save = ModelCheckpoint('CNN_3d_2x2_test_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
#    #opt = RAdam(total_steps=100, warmup_proportion=0.1, min_lr=1e-5)
#    model.compile(loss='mse', optimizer=opt, metrics=['mae'])
#    ### Running the model
#
#    
#    return model
#build_model_ = build_model()
#build_model_.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=2000, verbose=1)

In [ ]:
#def build_model_tune(hp):
#    
#    conv_size = hp.Int("conv_size", min_value=16, max_value=64, step=8)
#    kernel_size = hp.Int("kernel_size", min_value=2, max_value=12, step=2)
#    kernel_size_wide_n_tall = hp.Int("kernel_w_t_size", min_value=2, max_value=3, step=1)
#    dense_layer_size = hp.Int("dense_layer_size", min_value=64, max_value=256, step=64)
#    stride_val = hp.Int("stride_val", min_value=64, max_value=256, step=64)
#    maxpool_val =  hp.Int("maxpool_val", min_value=2, max_value=10, step=1)
#    
#    model = Sequential()
#    model.add(Conv3D(conv_size, kernel_size=(kernel_size, kernel_size_wide_n_tall, kernel_size_wide_n_tall),
#                     input_shape=(60, 3, 3, 4), padding='same', data_format="channels_last", strides = (1, 1, 1)))
#    model.add(LeakyReLU(alpha=0.3))
#    model.add(BatchNormalization())
#    ##model.add(Conv3D(32, (3, 3, 5), activation=LeakyReLU(alpha=0.3), padding='same'))
#    
#    for i in range(hp.Int("n_layers", 3, 7)):
#        model.add(Conv3D(conv_size, (kernel_size, kernel_size_wide_n_tall, kernel_size_wide_n_tall), padding='same'))
#        model.add(LeakyReLU(alpha=0.3))
#        model.add(BatchNormalization())
#    model.add(MaxPooling3D(pool_size=(maxpool_val, 3, 3)))
#    model.add(Flatten())
#    model.add(BatchNormalization())
#    model.add(Dense(dense_layer_size))
#    model.add(LeakyReLU(alpha=0.3))
#    model.add(BatchNormalization())
#    model.add(Dense(dense_layer_size))
#    model.add(LeakyReLU(alpha=0.3))
#    model.add(Dropout(hp.Float('dropout_value', min_value = 0.0, max_value = 0.5, step=0.25)))
#    model.add(Dense(1, activation='linear'))
#
#    ### Building the optimizer object
#    #sgd = optimizers.SGD(lr = 0.1) # 1.0 0.0001 0.01
#    opt = optimizers.Adam(lr = hp.Float('learning_rate', min_value = 0.0001, max_value = 0.1, step=0.0001)) 
#    ### Adding the loss function and optimizer to the model
#    #mcp_save = ModelCheckpoint('CNN_3d_2x2_test_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
#    #opt = RAdam(total_steps=100, warmup_proportion=0.1, min_lr=1e-5)
#    model.compile(loss='mse', optimizer=opt, metrics=['mae'])
#    ### Running the model
#
#    
#    return model
#LOG_DIR = "test_" + str(time.time())
#tuner = RandomSearch(build_model_tune,
#                    objective="val_mae",
#                     max_trials = 5,
#                     executions_per_trial = 1,
#                     directory = LOG_DIR)
#tuner.search_space_summary()
#tuner.search(x=X_train, y=Y_train,
#             epochs=100,
#             validation_split= 0.1,
#             batch_size=2500)

In [ ]:
from kerastuner import HyperModel
class HyperModel_555(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        conv_size = hp.Int("first_conv_size", min_value=16, max_value=64, step=1)
        kernel_size = hp.Int("kernel_size", min_value=3, max_value=4, step=1)
        dense_layer_size = hp.Int("dense_layer_size", min_value=32, max_value=256, step=32)
        max_pool_val = hp.Int("max_pool_val", min_value=1, max_value=10, step=1)

        model = Sequential()
        model.add(Conv3D(conv_size, kernel_size=(kernel_size, 2, 2),
                         input_shape=self.input_shape, padding='same', data_format="channels_last", strides = (1, 1, 1)))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        ##model.add(Conv3D(32, (3, 3, 5), activation=LeakyReLU(alpha=0.3), padding='same'))

        for i in range(hp.Int("n_layers", 3, 7)):
            model.add(Conv3D(hp.Int(f"conv_{i}_size", min_value=16, max_value=64, step=8), (kernel_size, 2, 2), padding='same'))
            model.add(LeakyReLU(alpha=0.3))
            model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(max_pool_val, 2, 2)))
        model.add(Flatten())
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(hp.Float('dropout_value', min_value = 0.0, max_value = 0.5, step=0.1)))
        
        model.add(Dense(1, activation='linear'))

        ### Building the optimizer object
        #sgd = optimizers.SGD(lr = 0.1) # 1.0 0.0001 0.01
        opt = optimizers.Adam(lr = hp.Float(
                    'learning_rate',
                    min_value=1e-7,
                    max_value=0.05,
                    default=0.00001,
                    step=1e-7)) 
           
        ### Adding the loss function and optimizer to the model
        #mcp_save = ModelCheckpoint('CNN_3d_2x2_test_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
        #opt = RAdam(total_steps=100, warmup_proportion=0.1, min_lr=1e-5)
        model.compile(loss='mse', optimizer=opt, metrics=['mae'])
        ### Running the model


        return model

In [ ]:
os.chdir(run_directory)
input_shape = (60, 2, 2, 4)
hypermodel = HyperModel_555(input_shape)

In [ ]:
LOG_DIR = "bayes_rep_1_"
max_trials_val = 200
# int(max_trials_val/10)
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mae',
            max_trials=max_trials_val,
            seed=1,
            executions_per_trial=2,
            num_initial_points = int(max_trials_val/10),
            directory = LOG_DIR)
tuner_bo.search(x=X_train, y=Y_train,
             epochs=100,
             batch_size=int(len(X_train)/2),
             validation_data=(X_test, Y_test))

In [ ]:
tuner_bo.results_summary()

In [ ]:
best_model = tuner_bo.get_best_models(num_models=10)

In [ ]:
best_model[0].summary()

In [ ]:
#loaded_model = keras.models.load_model("best_model_2_1_3ed_fns_5_mat")

In [ ]:
#loaded_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1000, batch_size=2500, verbose=1)

In [ ]:
pred_data = []
for model in best_model:
    pred_data.append(model.predict(X_train))

In [ ]:
for count, _ in enumerate(pred_data[0]):
    print_str = ''
    for count_model, __ in enumerate(pred_data):
        #print(pred_data[count_model][0][0])
        print_str += str(pred_data[count_model][count][0]) + "," 
    print(str(Y_train[count][0]) + "," + print_str)

In [ ]:
outputfile = open("out_2.csv", "w")
for count, _ in enumerate(pred_data[0]):
    print_str = ''
    for count_model, __ in enumerate(pred_data):
        #print(pred_data[count_model][0][0])
        print_str += str(pred_data[count_model][count][0]) + "," 
    outputfile.write(print_str+"\n")
outputfile.close()